In [1]:
import glob
import json
import os
import re
import time
import wget
import urllib.parse
import argparse


import numpy as np
import pandas as pd
import pubchempy as pcp


from pybatchclassyfire import *
from pandas import json_normalize
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import rdFMCS
from rdkit.Chem import PandasTools
def isNaN(string):
    return string != string

INFO:rdkit:Enabling RDKit 2021.09.4 jupyter extensions


In [5]:
spec_postproc(entry = "/Users/mahnoorzulfiqar/Downloads/New_ms2_spectra_endo_pos", Source="all")

In [4]:
def spec_postproc(entry, Source="all"):
    # currently only these subsets are removed from the names from GNPS
    matches = [
        "M+",
        "[M",
        "M-",
        "2M",
        "M*", 
        "20.0",
        "50.0",
        "30.0",
        "40.0",
        "60.0",
        "70.0",
        "eV",
        "Massbank",
        "Spectral",
        "Match",
        "to",
        "from",
        "NIST14",
        "MoNA",
        "[IIN-based:",
        "[IIN-based",
        "on:",
        "CCMSLIB00003136269]",
        "CollisionEnergy:"
    ]


    # Define scoring for all DBs
    def HMDB_Scoring(db, i):
        if (
            db["HMDBintScore"][i] >= 0.50
            and db["HMDBmzScore"][i] >= 0.50
            and db["HQMatchingPeaks"][i] / db["hQueryTotalPeaks"][i] >= 0.50
        ):
            return True
        else:
            return False

    def GNPS_Scoring(db, i):
        if (
            db["GNPSintScore"][i] >= 0.50
            and db["GNPSmzScore"][i] >= 0.50
            and db["GQMatchingPeaks"][i] / db["gQueryTotalPeaks"][i] >= 0.50
        ):
            return True
        else:
            return False

    def MB_Scoring(db, i):
        if (
            db["MBintScore"][i] >= 0.50
            and db["MBmzScore"][i] >= 0.50
            and db["MQMatchingPeaks"][i] / db["mQueryTotalPeaks"][i] >= 0.50
        ):
            return True
        else:
            return False
    # in case if we need HMDB later
    if os.path.exists(entry.split("/DS")[0] + "/hmdb_dframe_str.csv"):
        extract_smiles = pd.read_csv(entry.split("/DS")[0] + "/hmdb_dframe_str.csv", low_memory=False)
   

    msp_file = glob.glob(
        entry + "/spectral_dereplication" + "/*.csv"
    )
    


    if len(msp_file) > 0:

        if os.path.exists(msp_file[0]):

            msp = pd.read_csv(msp_file[0])
            msp["mbank_results_csv"] = np.nan
            msp["gnps_results_csv"] = np.nan
            msp["hmdb_results_csv"] = np.nan
            # enter the directory with /spectral_dereplication/ results

            # enter the directory with /spectral_dereplication/ results
            # GNPS Results
            if Source == "gnps" or Source == "all":

                # print(entry)
                # enter the directory with /spectral_dereplication/ results
                sub_dir = (
                    entry + "/spectral_dereplication/GNPS/"
                )

                if os.path.exists(sub_dir):
                    files = glob.glob(sub_dir + "/*.csv")
                    # print(files)
                    files = [item for item in files if 'proc' not in item]

                    for mz, row in msp.iterrows():
                        for fls_g in files:

                            if msp["id_X"][mz] in fls_g:
                                
                                gnps_df = pd.read_csv(fls_g)
                                if len(gnps_df) > 0:

                                    for i, row in gnps_df.iterrows():
                                        # if compound name is present

                                        if GNPS_Scoring(gnps_df, i):
                                            
                                            if not isNaN(
                                                gnps_df["GNPScompound_name"][i]
                                            ):
                                                # split if there is a gap in the names

                                                string_chng = gnps_df[
                                                    "GNPScompound_name"
                                                ][i].split(" ")

                                                # create an empty list
                                                newstr = []

                                                # for each part of the string in the names
                                                chng = []

                                                for j in range(
                                                    len(string_chng)
                                                ):
                                                    # check if the substrings are present in the matches and no - is present

                                                    if not any(
                                                        x in string_chng[j]
                                                        for x in matches
                                                    ):  # and not '-' == string_chng[j]:

                                                        # IF | and ! not in the substring
                                                        if (
                                                            "|"
                                                            not in string_chng[
                                                                j
                                                            ]
                                                            or "!"
                                                            not in string_chng[
                                                                j
                                                            ]
                                                        ):

                                                            newstr.append(
                                                                string_chng[j]
                                                            )
                                                        # if | present in the substring
                                                        elif (
                                                            "|"
                                                            in string_chng[j]
                                                        ):

                                                            # split the string
                                                            jlen = string_chng[
                                                                j
                                                            ].split("|")
                                                            # how many substrings are left now
                                                            lst = len(jlen) - 1
                                                            # append this to chng
                                                            chng.append(
                                                                jlen[lst]
                                                            )
                                                            break

                                                            # now append chng to newstr
                                                chng.append(" ".join(newstr))

                                                # save this as the correct name
                                                gnps_df.loc[
                                                    i, "corr_names"
                                                ] = chng[0]

                                                if not isNaN(
                                                    gnps_df["GNPSSMILES"][i]
                                                ):
                                                    if chng == "":
                                                        break
                                                    elif gnps_df["GNPSSMILES"][
                                                        i
                                                    ].isalpha():
                                                        s = pcp.get_compounds(
                                                            chng[0], "name"
                                                        )
                                                        if s:
                                                            for comp in s:
                                                                gnps_df[
                                                                    "GNPSSMILES"
                                                                ][
                                                                    i
                                                                ] = (
                                                                    comp.isomeric_smiles
                                                                )
                                                        else:
                                                            gnps_df[
                                                                "GNPSSMILES"
                                                            ][i] = ""
                                            else:
                                                gnps_df["GNPSSMILES"][i] = ""
                                        else:
                                            gnps_df.drop(
                                                [i], axis=0, inplace=True
                                            )
                                    gnps_df = gnps_df.drop_duplicates(
                                        subset=["GNPSSMILES"]
                                    )
                                    for k, row in gnps_df.iterrows():

                                        if isNaN(gnps_df["GNPSSMILES"][k]):

                                            if (
                                                "["
                                                in gnps_df["GNPScompound_name"][
                                                    k
                                                ].split(" ")[-1]
                                            ):
                                                string_chng = gnps_df[
                                                    "GNPScompound_name"
                                                ][k].split("[")
                                                # print(gnps_df['GNPScompound_name'][i])

                                                # keep_names = []
                                                for j in range(
                                                    len(string_chng) - 1
                                                ):
                                                    gnps_df.loc[
                                                        k, "corr_names"
                                                    ] == string_chng[j]
                                                    s = pcp.get_compounds(
                                                        string_chng[j], "name"
                                                    )

                                                    if s:
                                                        for comp in s:
                                                            gnps_df[
                                                                "GNPSSMILES"
                                                            ][
                                                                k
                                                            ] = (
                                                                comp.isomeric_smiles
                                                            )
                                                            gnps_df.loc[
                                                                k, "GNPSformula"
                                                            ] = (
                                                                comp.molecular_formula
                                                            )
                                                            gnps_df.loc[
                                                                k, "GNPSinchi"
                                                            ] = Chem.MolToInchi(
                                                                Chem.MolFromSmiles(
                                                                    comp.isomeric_smiles
                                                                )
                                                            )

                                                    else:
                                                        gnps_df["GNPSSMILES"][
                                                            k
                                                        ] = ""
                                                        gnps_df.loc[
                                                            k, "GNPSformula"
                                                        ] = ""
                                                        gnps_df.loc[
                                                            k, "GNPSinchi"
                                                        ] = ""
                                        if not isNaN(gnps_df["GNPSSMILES"][k]):
                                            try:
                                                sx = pcp.get_compounds(
                                                    gnps_df["GNPSSMILES"][k],
                                                    "smiles",
                                                )
                                                gnps_df.loc[
                                                    k, "GNPSinchi"
                                                ] = Chem.MolToInchi(
                                                    Chem.MolFromSmiles(
                                                        comp.isomeric_smiles
                                                    )
                                                )
                                                if sx:
                                                    sx = str(sx)
                                                    comp = pcp.Compound.from_cid(
                                                        [
                                                            int(x)
                                                            for x in re.findall(
                                                                r"\b\d+\b", sx
                                                            )
                                                        ]
                                                    )
                                                    gnps_df.loc[
                                                        k, "GNPSformula"
                                                    ] = comp.molecular_formula

                                            except Exception:
                                                gnps_df.loc[
                                                    k, "GNPSformula"
                                                ] = ""
                                                gnps_df.loc[k, "GNPSinchi"] = ""

                                gnps_df = gnps_df.dropna(axis=0, how="all")
                                csvname = (
                                    (os.path.splitext(fls_g)[0])
                                    + "proc"
                                    + ".csv"
                                )

                                msp.loc[
                                    mz, "gnps_results_csv"
                                ] = csvname
                                
                                if not os.path.exists(csvname):
                                    #print("this is wrong?")
                                    #print(csvname)
                                    #print(os.path.splitext(fls_g)[0])
                                    gnps_df.to_csv(csvname)


            msp.to_csv(msp_file[0])
            # HMDB Results
            if Source == "hmdb" or Source == "all":
                sub_dir = (
                    entry + "/spectral_dereplication/HMDB/"
                )
                if os.path.exists(sub_dir):
                    files = glob.glob(sub_dir + "/*.csv")
                    files = [item for item in files if 'proc' not in item]
                    if os.path.exists(sub_dir):
                        # print(files)
                        for mz, row in msp.iterrows():
                            #print(mz)
                            # print(msp["id_X"][mz])
                            for fls_h in files:
                                 if msp["id_X"][mz] in fls_h:
                                        hmdb_df = pd.read_csv(fls_h)

                                        if len(hmdb_df) > 0:
                                            if "HMDBSMILES" in hmdb_df.columns:
                                                #print(hmdb_df)
                                                for i, row in hmdb_df.iterrows():
                                                # if compound name is present
                                                    if not HMDB_Scoring(hmdb_df, i):
                                                        hmdb_df.drop(i, inplace=True)
                                                hmdb_df = hmdb_df.drop_duplicates(
                                                    subset=["HMDBSMILES"]
                                                )


                                                csvname = (
                                                    (os.path.splitext(fls_h)[0])
                                                    + "proc"
                                                    + ".csv"
                                                )  
                                                msp.loc[
                                                    mz, "hmdb_results_csv"
                                                ] = csvname

                                                if not os.path.exists(csvname):
                                                    hmdb_df.to_csv(csvname) 
                                            else:    
                                                # merge on basis of id, frame and hmdb result files
                                                SmilesHM = pd.merge(
                                                    hmdb_df,
                                                    extract_smiles,
                                                    left_on=hmdb_df.HMDBcompoundID,
                                                    right_on=extract_smiles.DATABASE_ID,
                                                )
                                                hmdb_df["HMDBcompoundID"] = np.nan
                                                hmdb_df["HMDBSMILES"] = np.nan
                                                hmdb_df["HMDBformula"] = np.nan
                                                hmdb_df["HMDBcompound_name"] = np.nan
                                                for i, row in hmdb_df.iterrows():
                                                    #print(i)
                                                    # if compound name is present
                                                    if HMDB_Scoring(hmdb_df, i):

                                                        #hmdb_df.drop(i, inplace=True)
                                                        for j, row in SmilesHM.iterrows():
                                                            #print("SmilesHM")
                                                            # where index for both match, add the name and SMILES
                                                            if (
                                                                hmdb_df["HMDBcompoundID"][i]
                                                                == SmilesHM[
                                                                    "HMDBcompoundID"
                                                                ][j]
                                                            ):
                                                                hmdb_df.loc[
                                                                    i, "HMDBSMILES"
                                                                ] = SmilesHM["SMILES"][
                                                                    j
                                                                ]  # add SMILES
                                                                hmdb_df.loc[
                                                                    i, "HMDBcompound_name"
                                                                ] = SmilesHM[
                                                                    "GENERIC_NAME"
                                                                ][
                                                                    j
                                                                ]  # add name
                                                                hmdb_df.loc[
                                                                    i, "HMDBformula"
                                                                ] = SmilesHM["FORMULA"][
                                                                    j

                                                                ]
                                                                #print(hmdb_df["HMDBSMILES"][i])
                #                             
                                            #print(hmdb_df)
#                                         hmdb_df = hmdb_df.drop_duplicates(
#                                              subset=["HMDBSMILES"]
#                                          )
                                        csvname = (
                                            (os.path.splitext(fls_h)[0])
                                            + "proc"
                                            + ".csv"
                                        )  
                                        msp.loc[
                                            mz, "hmdb_results_csv"
                                        ] = csvname

                                        if not os.path.exists(csvname):
                                            hmdb_df.to_csv(csvname) 
            msp.to_csv(msp_file[0])
            # MASSBANK Results

            # enter the directory with /spectral_dereplication/ results
            if Source == "mbank" or Source == "all":

                sub_dir = (
                    
                    entry
                    + "/spectral_dereplication/MassBank/"
                )
                if os.path.exists(sub_dir):
                    files = glob.glob(sub_dir + "/*.csv")
                    files = [item for item in files if 'proc' not in item]
                    for mz, row in msp.iterrows():
                        # print(msp["id_X"][mz])
                        for fls_m in files:
                            if msp["id_X"][mz] in fls_m:
                                mbank_df = pd.read_csv(fls_m)
                                if len(mbank_df) > 0:

                                    for i, row in mbank_df.iterrows():
                                        # if compound name is present
                                         if not MB_Scoring(mbank_df, i):
                                            mbank_df.drop(i, inplace=True)
                                mbank_df = mbank_df.drop_duplicates(
                                    subset=["MBSMILES"]
                                )
                                csvname = (
                                    (os.path.splitext(fls_m)[0])
                                    + "proc"
                                    + ".csv"
                                )

                                msp.loc[
                                    mz, "mbank_results_csv"
                                ] = csvname

                                if not os.path.exists(csvname):

                                    mbank_df.to_csv(csvname)
            msp.to_csv(msp_file[0])

In [17]:
def gnpsMNvsgnpsMAW(entry, mn_dir, name):
    
    """gnpsMNvsgnpsMAW checks with tanimoto similarity score, whether
    results from MAW GNPS and GNPS MN Masst results give same candidate

    Parameters:
    input_dir = input directory where you have stored the cytoscape file
    from GNPS MN results and have exported edge and node tables from cytoscape
    These two csv egde and node files must have "edge" and "node" in their name

    Returns:
    GNPS results with cluster index named
    GNPS MN results with a confirmation column if MAW detected same candidate,
    file named:

    Usage:
    gnpsMNvsgnpsMAW(mn_dir)
    """
    # extract files with edges from MN results
    GMNfile_edge = [f for f in os.listdir(mn_dir) if "edge" in f]
    # extract files with nodes from MN results
    GMNfile_node = [f for f in os.listdir(mn_dir) if "node" in f]
    # read the files
    GMNdf_node = pd.read_csv(mn_dir + "/" + GMNfile_node[0])
    GMNdf_edge = pd.read_csv(mn_dir + "/" + GMNfile_edge[0])
    # extract only important columns from both csv files
    GMNdf_node = GMNdf_node[
        [
            "precursor mass",
            "RTMean",
            "UniqueFileSources",
            "charge",
            "cluster index",
            "componentindex",
            "Compound_Name",
            "Smiles",
            "SpectrumID",
        ]
    ]
    GMNdf_edge = GMNdf_edge[
        ["cosine_score", "EdgeAnnotation", "node1", "node2", "mass_difference"]
    ]
    # rename node1 to cluster index to merge nodes and edges results from MN
    GMNdf_edge = GMNdf_edge.rename(columns={"node1": "cluster index"})
    GMNdf = pd.merge(GMNdf_node, GMNdf_edge, on="cluster index")
    GMNdf.to_csv(mn_dir + "/mergedN&E.csv")
    # Read results obtained from scoring_spec, named input_dir/MetabolomicsResults/scoredSpecDB.csv
    SDB = pd.read_csv(entry + "/mergedResults-with-one-Candidates.csv")
    # from GNPS MAW results and GNPS MN results, calculate how many MAW results are same as MN:
    for i, row in SDB.iterrows():
        for j, row in GMNdf.iterrows():
             if not isNaN(SDB["SMILES"][i]) and not isNaN(GMNdf["Smiles"][j]):
                if name in GMNdf["UniqueFileSources"][j]: 
                    SKms = [
                        Chem.MolFromSmiles(SDB["SMILES"][i]),
                        Chem.MolFromSmiles(GMNdf["Smiles"][j]),
                    ]
                    SKfps = [
                        AllChem.GetMorganFingerprintAsBitVect(x, 2, nBits=2048)
                        for x in SKms
                    ]
                    SKtn = DataStructs.FingerprintSimilarity(SKfps[0], SKfps[1])
                    if SKtn >= 0.99:
                        print(SKtn)
                        print(GMNdf["Smiles"][j])
                        SDB['AnnotationSources'][i] = SDB['AnnotationSources'][i] + "|GNPSMN"
                        SDB['MSILevel'][i] = 2.0
                        print(SDB['AnnotationSources'][i])
                    #else:

    SDB.to_csv(entry + "/mergedResults-with-one-Candidates.csv")

In [18]:
path = "/Users/mahnoorzulfiqar/OneDriveUNI/MAW-Diatom/SmarinoiRun1"
file = os.listdir(path)
folders = [x for x in os.listdir(path) if x.startswith('DS')]
folders2 = [x for x in folders if not '.mzML' in x]
folders2

['DS_201124_SC_full_PRM_pos_03',
 'DS_201124_SC_full_PRM_pos_04',
 'DS200309_Scost_QC_70k_neg_PRM',
 'DS_201124_SC_full_PRM_pos_05',
 'DS_201124_SC_full_PRM_pos_02',
 'DS_201124_SC_full_PRM_neg_03',
 'DS_201124_SC_full_PRM_neg_04',
 'DS200309_Scost_QC_70k_pos_PRM',
 'DS_201124_SC_full_PRM_neg_05',
 'DS_201124_SC_full_PRM_neg_02',
 'DS_201124_SC_full_PRM_neg_10',
 'DS_201124_SC_full_PRM_pos_10',
 'DS_201124_SC_full_PRM_pos_07',
 'DS_201124_SC_full_PRM_pos_09',
 'DS_201124_SC_full_PRM_pos_08',
 'DS_201124_SC_full_PRM_pos_01',
 'DS_201124_SC_full_PRM_pos_06',
 'DS_201124_SC_full_PRM_neg_07',
 'DS_201124_SC_full_PRM_neg_09',
 'DS_201124_SC_full_PRM_neg_08',
 'DS_201124_SC_full_PRM_neg_01',
 'DS_201124_SC_full_PRM_neg_06']

In [21]:
mn_dir = "/Users/mahnoorzulfiqar/OneDriveUNI/MAW-Diatom/Molecular-Networking/GNPS/First-Run"

In [22]:
for i in folders2:
    entry = path + "/" + i
    name = i
    gnpsMNvsgnpsMAW(entry, mn_dir, name)

1.0
OC(=O)C(N)Cc(c1)ccc(O)c1
SIRIUS|MassBank|GNPS|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN
1.0
CCCC(=O)O[C@H](CC([O-])=O)C[N+](C)(C)C
GNPS|GNPSMN|GNPSMN|GNPSMN
1.0
C(C[C@@H](C(=O)O)N)CN=C(N)N
SIRIUS|GNPS|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN
1.0
C(C[C@@H](C(=O)O)N)CN=C(N)N
SIRIUS|GNPS|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN
1.0
C(C[C@@H](C(=O)O)N)CN=C(N)N
SIRIUS|GNPS|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN
1.0
CCCCCC/C=C\\CCCCCCCC(=O)OCC(CO)O
GNPS|GNPSMN|GNPSMN|GNPSMN
1.0
CCC(=O)O[C@H](CC(O)=O)C[N+](C)(C)C
SIRIUS|GNPS|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN
1.0
CCC(=O)O[C@H](CC(O)=O)C[N+](C)(C)C
SIRIUS|GNPS|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN
1.0
O=C(O)[C@H](O)[C@@H](O)[C@H](O)[C@H](O)CO
SIRIUS|GNPS|GNPSMN|GNPSMN|GNPSMN
1.0
OC(=O)C(N)Cc(c1)ccc(O)c1
MassBank|GNPS|GNPSMN|GNPSMN|GNPSMN
1.0
CCCCCCCCCCCCCCCC(=O)OC[C@H](COP(=O)([O-])OCC[N+](C)(C)C)O
GNPS|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNPSMN|GNP